In [ ]:
import pandas as pd
import geopandas as gpd
import os

### Goal - get Food Insecurity Classification units with spatial data

## Step 1: Spatial Data Extraction
It is recommended to extract spatial data separately to reduce the downloaded file size

In [ ]:
# Setup base url for all queries
base_url = "https://fdw.fews.net"

In [ ]:
# Go to https://fdw.fews.net/data-explorer/spatial?unit_type=fsc_admin%2Cfsc_admin_lhz%2Cfsc_lhz%2Cfsc_rm_admin
# then select all units (checkbox next to the Country table header)
# then select 'geojson' format at the bottom and click on the API link to copy the link.
# Currently FDE does not include IDP camps into FCS units, adding it manually.
# spatial_url variable is already populated with the result of the actions above.

spatial_url = "/api/feature/?format=geojson&unit_type=fsc_admin&unit_type=fsc_admin_lhz&unit_type=fsc_lhz&unit_type=fsc_rm_admin&unit_type=idp_camp"

In [ ]:
# Create a geodataframe of all available FIC units over all time periods
# Takes around 60-90 seconds to read
spatial_dataframe = gpd.read_file(base_url + spatial_url)
spatial_dataframe.head()

In [ ]:
spatial_dataframe.info()

## Step 2: Food Insecurity Data Points Extraction

In [ ]:
# Go to https://fdw.fews.net/data-explorer/ipcFic?geographic_unit=902%2C904%2C905%2C903%2C901 (data might take some time to load)
# then select 'json' format at the bottom and click on the API link to copy the link
# paste the copied url to fic_url variable below (already populated).
# NOTE `&fields=simple` parameter need to be removed to allow fnid field to be available.
# NOTE selection applies 'Best available' filter that returns all historical Current Situation data
# as well as the most recent Medium or Near Term projections if available.

food_security_url = "/api/ipcphase/?format=json&country_code=BI&country_code=DJ&country_code=ET&country_code=KE&country_code=RW&country_code=SO&country_code=SS&country_code=SD&country_code=UG&country_code=CO&country_code=SV&country_code=GT&country_code=HT&country_code=HN&country_code=NI&country_code=VE&country_code=AF&country_code=LB&country_code=YE&country_code=AO&country_code=CD&country_code=LS&country_code=MG&country_code=MW&country_code=MZ&country_code=ZM&country_code=ZW&country_code=BF&country_code=CM&country_code=CF&country_code=TD&country_code=LR&country_code=ML&country_code=MR&country_code=NE&country_code=NG&country_code=SL&country_code=TG&preference=best"


In [ ]:
# Create a dataframe of all available FIC data over all time periods
# Takes around 30 seconds to read
fic_dataframe = pd.read_json(base_url + food_security_url)
fic_dataframe.head()

In [ ]:
fic_dataframe.info()

## Step 3: Merge Both DataFrames

In [ ]:
dataframe = spatial_dataframe[['fnid', 'full_name', 'geometry']].merge(
    fic_dataframe, on='fnid', how='right'
)
dataframe.info()

## Step 4: Convert data into FlatGeobuf format

In [ ]:
# Remove not collected values and empty geometries
# dataframe = dataframe[~dataframe['value'].isnull()].reset_index(drop=True)
dataframe = dataframe[~dataframe['geometry'].isnull()].reset_index(drop=True)

In [ ]:
# Ensure the crs is correct
fgb_dataframe = dataframe.set_crs(4326)

# Define columns that will be stored in the fgb file
columns = ['fnid', 'full_name', 'geometry', 'source_organization',
       'source_document', 'country', 'country_code', 'geographic_group',
       'fewsnet_region', 'unit_type', 'classification_scale', 'scenario_name',
       'preference_rating', 'is_allowing_for_assistance', 'reporting_date', 'projection_start',
       'projection_end', 'status', 'value', 'description', 'id']

In [ ]:
# Save files by region and collection date
for (fewsnet_region, reporting_date), g in fgb_dataframe[columns].groupby(["fewsnet_region","reporting_date"]):
    out_dir = os.path.join("all_fic", str(fewsnet_region))
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{fewsnet_region}_{reporting_date}.fgb")
    g.to_file(out_path, driver="FlatGeobuf")